In [1]:
import os
import logging
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import torch
import spacy
from transformers import AutoTokenizer, AutoModel, pipeline
from neo4j import GraphDatabase

from typing import Any, Dict
from pydantic import BaseModel, validator, Field
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Function to load and preprocess data from various sources
def load_data():
    # Load CSV files
    united_dates_locations = pd.read_csv("data/united_dates_locations.csv")
    alliance_dates_locations = pd.read_csv("data/alliance_dates_locations.csv")
    air_canada_dates_locations = pd.read_csv("data/air_canada_dates_locations.csv")

    # Load text files
    with open("data/united_aircraft_details.txt", "r") as file:
        united_aircraft_details_content = file.read().split('\n\n')
    with open("data/alliance_aircraft_details.txt", "r") as file:
        alliance_aircraft_details_content = file.read().split('\n\n')
    with open("data/air_canada_aircraft_details.txt", "r") as file:
        air_canada_aircraft_details_content = file.read().split('\n\n')

    # Extract text from PDF files
    def extract_text_from_pdf(pdf_path):
        text = []
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page_num in range(len(reader.pages)):
                text.append(reader.pages[page_num].extract_text())
        return "\n".join(text)

    united_pdf_content = extract_text_from_pdf("data/united_accident_outcomes.pdf")
    alliance_pdf_content = extract_text_from_pdf("data/alliance_accident_outcomes.pdf")
    air_canada_pdf_content = extract_text_from_pdf("data/air_canada_accident_outcomes.pdf")

    # Combine data
    united_data = united_dates_locations['summary'].tolist() + united_aircraft_details_content + [united_pdf_content]
    alliance_data = alliance_dates_locations['summary'].tolist() + alliance_aircraft_details_content + [alliance_pdf_content]
    air_canada_data = air_canada_dates_locations['summary'].tolist() + air_canada_aircraft_details_content + [air_canada_pdf_content]

    return united_data, alliance_data, air_canada_data

# Load the data
united_data, alliance_data, air_canada_data = load_data()

In [3]:
# Configure logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# Load models
nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
rel_extractor = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Load Hugging Face local embeddings
hugging_face_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize the generative model
generative_model = Ollama(model="gemma:7b")

# Define a prompt template
prompt_template = PromptTemplate(
    template="""
    You are an AI assistant that specializes in providing detailed information about airline accidents. 
    When given a query about a specific flight, you should:

    1. Identify the flight number and any other relevant details from the query.
    2. Retrieve specific information about the flight from the provided context, including any relevant accidents or incidents.
    3. Summarize the information in a clear and concise manner within one sentence.
    4. If there are multiple incidents related to the flight, provide details on each incident separately.
    5. Ensure the response is focused on the specific flight mentioned in the query.

    Use the provided context to generate the response and avoid including unrelated information.

    Context:
    {context}
    """,
    input_variables=["context"]
)

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Initialize document stores
class InMemoryDocstore:
    def __init__(self, documents):
        self.documents = documents

    def get_document(self, doc_id):
        return self.documents.get(doc_id, None)

In [5]:
def vectorize_and_index(data_list, index_path):
    embeddings = np.vstack([embed_text(text) for text in data_list])
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.astype('float32'))
    faiss.write_index(index, index_path)

vectorize_and_index(united_data, "faiss_indexes/united_faiss.index")
vectorize_and_index(alliance_data, "faiss_indexes/alliance_faiss.index")
vectorize_and_index(air_canada_data, "faiss_indexes/air_canada_faiss.index")

# Initialize document stores
united_docstore = InMemoryDocstore({str(i): Document(page_content=text, metadata={"doc_id": f"united_{i}"}) for i, text in enumerate(united_data)})
alliance_docstore = InMemoryDocstore({str(i): Document(page_content=text, metadata={"doc_id": f"alliance_{i}"}) for i, text in enumerate(alliance_data)})
air_canada_docstore = InMemoryDocstore({str(i): Document(page_content=text, metadata={"doc_id": f"air_canada_{i}"}) for i, text in enumerate(air_canada_data)})

def create_faiss_retriever(index_path, docstore, data_list):
    index = faiss.read_index(index_path)
    index_to_docstore_id = {i: str(i) for i in range(len(data_list))}
    return FAISS(embedding_function=embed_text, index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

united_faiss_retriever = create_faiss_retriever("faiss_indexes/united_faiss.index", united_docstore, united_data)
alliance_faiss_retriever = create_faiss_retriever("faiss_indexes/alliance_faiss.index", alliance_docstore, alliance_data)
air_canada_faiss_retriever = create_faiss_retriever("faiss_indexes/air_canada_faiss.index", air_canada_docstore, air_canada_data)

In [6]:
class GraphDBClient:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record for record in result]

In [7]:
def query_knowledge_graph(client, keyword):
    query = """
    MATCH (e:Entity)-[r]->(related:Entity)
    WHERE e.name =~ $name OR related.name =~ $name
    RETURN e.name AS entity, type(r) AS relationship, related.name AS related_entity
    """
    try:
        result = client.query(query, parameters={"name": f"(?i).*{keyword}.*"})
        return [record for record in result]
    except Exception as e:
        logger.error(f"Error querying knowledge graph: {e}")
        return []

In [8]:
class CustomMultiRetriever(BaseModel):
    faiss_retrievers: Dict[str, FAISS] = Field(..., description="Dictionary of FAISS retrievers")
    knowledge_graph_client: Any

    @validator('faiss_retrievers')
    def check_faiss_retrievers(cls, value):
        if not value:
            raise ValueError("FAISS retrievers cannot be empty")
        return value

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query):
        query_vector = embed_text(query).reshape(1, -1)
        results = {}
        for airline, retriever in self.faiss_retrievers.items():
            distances, indices = retriever.index.search(query_vector, k=10)
            docs = [retriever.docstore.get_document(str(i)) for i in indices[0] if retriever.docstore.get_document(str(i)) is not None]
            results[airline] = docs
        return results


/tmp/ipykernel_1950/726269644.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('faiss_retrievers')


In [9]:
def integrate_graph_and_docs(graph_results, retrieved_docs):
    integrated_context = "Retrieved Context:\n"
    if graph_results:
        for result in graph_results:
            entity = result.get('entity', 'Unknown Entity')
            relationship = result.get('relationship', 'Unknown Relationship')
            related_entity = result.get('related_entity', 'Unknown Related Entity')
            integrated_context += f"{entity} {relationship} {related_entity}. "
    else:
        integrated_context += "No relevant data found in the knowledge graph.\n"
    
    if retrieved_docs:
        integrated_context += "\nFrom FAISS:\n"
        for doc in retrieved_docs:
            integrated_context += doc.page_content + "\n"
    else:
        integrated_context += "No relevant data found in the FAISS retrievers.\n"
    
    return integrated_context

In [10]:
def extract_entities(query):
    doc = nlp(query)
    keywords = [ent.text for ent in doc.ents if ent.label_ in ("ORG", "GPE", "NORP", "PRODUCT", "EVENT")]
    return keywords

In [28]:
def process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain):
    try:
        entities = extract_entities(user_query)
        logger.info(f"Extracted entities: {entities}")

        graph_context = ""
        for entity in entities:
            graph_results = query_knowledge_graph(graphdb_client, entity)
            logger.info(f"Graph results for entity {entity}: {graph_results}")
            if graph_results:
                graph_context += integrate_graph_and_docs(graph_results, [])
            else:
                graph_context += f"No relevant data found in the knowledge graph for entity: {entity}.\n"

        faiss_docs = custom_multi_retriever.get_relevant_documents(user_query)

        final_context = graph_context + " ".join([doc.page_content for docs in faiss_docs.values() for doc in docs])

        response = llm_chain({"context": final_context})

        # Extract doc_id from retrieved documents
        retrieved_doc_ids = {key: [doc.metadata["doc_id"] for doc in docs] for key, docs in faiss_docs.items()}
        return response['text'], retrieved_doc_ids
    except Exception as e:
        logger.error(f"Error processing query: {e}")
        raise



In [29]:
# Initialize connection to Neo4j
graphdb_client = GraphDBClient(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), password=os.getenv('NEO4J_PASSWORD'))

custom_multi_retriever = CustomMultiRetriever(
    faiss_retrievers={
        "United Airlines": united_faiss_retriever,
        "Alliance Airlines": alliance_faiss_retriever,
        "Air Canada": air_canada_faiss_retriever
    },
    knowledge_graph_client=graphdb_client
)

# Create the LLMChain
llm_chain = LLMChain(prompt=prompt_template, llm=generative_model)

user_query = "Tell me about what aircraft FLIGHT 624 faced accident"
response,sample_doc_ret = process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain)
print(f"Response: {response}")


Response: **Query:** Provide information about Air Canada Flight 624.

**Response:** Air Canada Flight 624 suffered a hard landing at Halifax Stanfield International Airport in heavy snow on approach, resulting in severe damage to the aircraft. All passengers and crew evacuated safely, with 25 people sustaining minor injuries.


In [30]:
print(sample_doc_ret)

{'United Airlines': ['united_6', 'united_5', 'united_4', 'united_7', 'united_3', 'united_2', 'united_1', 'united_0'], 'Alliance Airlines': ['alliance_4', 'alliance_5', 'alliance_6', 'alliance_7', 'alliance_3', 'alliance_0', 'alliance_1', 'alliance_2'], 'Air Canada': ['air_canada_5', 'air_canada_4', 'air_canada_6', 'air_canada_7', 'air_canada_3', 'air_canada_2', 'air_canada_0', 'air_canada_1']}


In [31]:
#user_query = "Tell me about what aircraft FLIGHT 328"
#response = process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain)
#print(f"Response: {response}")

In [32]:
from sklearn.metrics import precision_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

def precision_at_k(retrieved_docs, relevant_docs, k=10):
    relevant_set = set(relevant_docs)
    retrieved_k = retrieved_docs[:k]
    retrieved_set = set(retrieved_k)
    intersection = retrieved_set.intersection(relevant_set)
    precision = len(intersection) / len(retrieved_k) if retrieved_k else 0
    return precision

def mean_reciprocal_rank(retrieved_docs, relevant_docs):
    for i, doc in enumerate(retrieved_docs):
        if doc in relevant_docs:
            return 1 / (i + 1)
    return 0

def ndcg_at_k(retrieved_docs, relevant_docs, k=10):
    dcg = 0
    for i, doc in enumerate(retrieved_docs[:k]):
        if doc in relevant_docs:
            dcg += 1 / np.log2(i + 2)
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant_docs), k)))
    return dcg / idcg if idcg > 0 else 0

def bleu_score(reference, hypothesis):
    smoothing_function = SmoothingFunction().method1
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()
    return sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing_function)

# Function to evaluate multiple metrics
def evaluate_performance(retrieved_docs, relevant_docs, generated_response, reference_response, k=10):
    precision = precision_at_k(retrieved_docs, relevant_docs, k)
    mrr = mean_reciprocal_rank(retrieved_docs, relevant_docs)
    ndcg = ndcg_at_k(retrieved_docs, relevant_docs, k)
    bleu = bleu_score(reference_response, generated_response)
    return precision, mrr, ndcg, bleu


In [33]:
test_queries = [
    {
        "query": "Tell me about United Airlines Flight 232",
        "relevant_docs": ["united_0","united_1","united_2"],
        "expected_response": "United Airlines Flight 232 suffered an uncontained engine failure, leading to the loss of all hydraulic systems. The aircraft broke apart upon landing. Out of 296 passengers and crew, 111 were killed, and 185 survived."
    },
    {
        "query": "Details of United Airlines Flight 93",
        "relevant_docs": ["united_3","united_4","united_5"],
        "expected_response": "United Airlines Flight 93 was hijacked during the September 11 attacks. The plane crashed into a field, killing all 44 people on board. The passengers' actions prevented the aircraft from reaching its intended target."
    },
    {
        "query": "Incident with United Airlines Flight 328",
        "relevant_docs": ["united_6","united_7","united_8"],
        "expected_response": "United Airlines Flight 328 experienced a right engine failure shortly after takeoff. The aircraft returned to Denver International Airport for an emergency landing with no injuries."
    },
    {
        "query": "Tell me about Air Canada Flight 797",
        "relevant_docs": ["air_canada_0","air_canada_1","air_canada_5"],
        "expected_response": "Air Canada Flight 797 had a fire start in the lavatory while in flight. The aircraft landed, but the fire rapidly spread, resulting in the deaths of 23 passengers. 21 passengers and all crew members survived."
    },
    {
        "query": "Details of Air Canada Flight 624",
        "relevant_docs": ["air_canada_0","air_canada_1","air_canada_2"],
        "expected_response": "Air Canada Flight 624 struck power lines and an antenna array upon approach in heavy snow at Halifax Stanfield International Airport. All passengers and crew evacuated safely, with 25 people sustaining minor injuries."
    },
    {
        "query": "Incident with Air Canada Flight 759",
        "relevant_docs": ["air_canada_3","air_canada_4","air_canada_5"],
        "expected_response": "Air Canada Flight 759 was cleared to land on a runway that was closed for maintenance at San Francisco International Airport. The aircraft almost landed on a taxiway, but the incident was averted, and it landed safely on a different runway."
    },
    {
        "query": "Tell me about Alliance Airlines Flight 764",
        "relevant_docs": ["alliance_0","alliance_1","alliance_2"],
        "expected_response": "Alliance Airlines Flight 764 suffered a hydraulic system failure shortly after takeoff. The aircraft returned to Brisbane and landed safely without any injuries."
    },
    {
        "query": "Details of Alliance Airlines Flight 850",
        "relevant_docs": ["alliance_3","alliance_4","alliance_5"],
        "expected_response": "Alliance Airlines Flight 850 struck a flock of birds during final approach to Perth Airport, causing significant damage to the right engine. The aircraft landed safely with no injuries."
    },
    {
        "query": "Incident with Alliance Airlines Flight 982",
        "relevant_docs": ["alliance_6","alliance_7","alliance_8"],
        "expected_response": "Alliance Airlines Flight 982 encountered severe turbulence while descending towards Melbourne. Several passengers were injured, but the aircraft landed safely."
    }
]

results = []

# Initialize connection to Neo4j
graphdb_client = GraphDBClient(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), password=os.getenv('NEO4J_PASSWORD'))

# Evaluate Performance
for test_case in test_queries:
    user_query = test_case["query"]
    relevant_docs = test_case["relevant_docs"]
    reference_response = test_case["expected_response"]

    generated_response, retrieved_docs = process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain)
    print(retrieved_docs)

    # Flatten the retrieved documents
    retrieved_doc_ids = [doc_id for doc_list in retrieved_docs.values() for doc_id in doc_list]

    precision, mrr, ndcg, bleu = evaluate_performance(retrieved_doc_ids, relevant_docs, generated_response, reference_response)
    
    results.append({
        "query": user_query,
        "precision@10": precision,
        "mrr": mrr,
        "ndcg@10": ndcg,
        "bleu": bleu,
        "generated_response": generated_response
    })

{'United Airlines': ['united_4', 'united_7', 'united_5', 'united_6', 'united_3', 'united_1', 'united_0', 'united_2'], 'Alliance Airlines': ['alliance_7', 'alliance_6', 'alliance_5', 'alliance_4', 'alliance_3', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_4', 'air_canada_5', 'air_canada_7', 'air_canada_2', 'air_canada_3', 'air_canada_0', 'air_canada_1']}


/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


{'United Airlines': ['united_5', 'united_4', 'united_7', 'united_3', 'united_6', 'united_1', 'united_0', 'united_2'], 'Alliance Airlines': ['alliance_6', 'alliance_7', 'alliance_4', 'alliance_5', 'alliance_3', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_4', 'air_canada_7', 'air_canada_5', 'air_canada_3', 'air_canada_2', 'air_canada_0', 'air_canada_1']}


/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/codespace/.python/current/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


KeyboardInterrupt: 

In [ ]:
# Print Evaluation Results
for result in results:
    print(f"Query: {result['query']}")
    print(f"Precision@10: {result['precision@10']}")
    print(f"MRR: {result['mrr']}")
    print(f"nDCG@10: {result['ndcg@10']}")
    print(f"BLEU: {result['bleu']}")
    print(f"Generated Response: {result['generated_response']}\n")